In [1]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-7043.csv")

In [2]:
df=df.drop(['customerID'], axis=1) 
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)




#BOX-COX
lemda=0.5
num_df=(num_df**lemda)
num_df=num_df-1
num_df[num_df < 0]=0
num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
#num_df=round(np.log(num_df.add(1)),2)
#num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log


result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)


result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [3]:
X.shape

(7043, 19)

In [4]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
1,tenure,2911.079764
18,TotalCharges,1616.483021
15,Contract,1115.780167
9,OnlineSecurity,551.611529
12,TechSupport,523.303866
2,MonthlyCharges,362.118432
10,OnlineBackup,230.086520
11,DeviceProtection,191.303140
5,Dependents,133.036443
16,PaperlessBilling,105.680863


In [5]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(7043, 15)

In [6]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

3952 1222 493 1376
pod:  0.7362225789192082
pof:  0.23618090452261306
AUC:  0.7500208371982976


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits
{'var_smoothing': 1.519911082952933e-06}
GaussianNB(var_smoothing=1.519911082952933e-06)
0.7864527886207712


In [8]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=3.5111917342151277e-07)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4266 908 673 1196
pod:  0.6399143927233815
pof:  0.17549284885968303
AUC:  0.7322107719318492
accuracy:  0.7755217946897629


In [9]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
{'C': 5, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=5, multi_class='ovr', penalty='l1', solver='liblinear')
0.8022144987105093


In [11]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', verbose=0,
                   warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4682 492 901 968
pod:  0.5179240235420011
pof:  0.09509083880943177
AUC:  0.7114165923662846
accuracy:  0.8022149652136874


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 10 folds for each of 16 candidates, totalling 160 fits
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance')
0.7583403852353321


In [13]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='distance', n_neighbors=19 )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4833 341 1361 508
pod:  0.27180310326377743
pof:  0.06590645535369154
AUC:  0.6029483239550429
accuracy:  0.7583416157887264


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
{'bootstrap': True, 'max_depth': 110, 'max_features': 3, 'min_samples_leaf': 5, 'min_samples_split': 12, 'n_estimators': 100}
RandomForestClassifier(max_depth=110, max_features=3, min_samples_leaf=5,
                       min_samples_split=12)
0.805620164410058


In [15]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=300)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4728 446 953 916
pod:  0.4901016586409845
pof:  0.08620023192887515
AUC:  0.7019507133560547
accuracy:  0.8013630555161153


In [16]:
# Decision Tree
import time
start = time.time()
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)
end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
{'criterion': 'entropy', 'splitter': 'random'}
DecisionTreeClassifier(criterion='entropy', splitter='random')
0.7356223807221148
Time:  0.33646106719970703


In [17]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifier =DecisionTreeClassifier(criterion='entropy', splitter='random')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4176 998 900 969
pod:  0.5184590690208668
pof:  0.1928875144955547
AUC:  0.6627857772626561
accuracy:  0.7305125656680392


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.28181818181818186, 'n_estimators': 10, 'subsample': 0.95}
GradientBoostingClassifier(learning_rate=0.2, max_depth=8, max_features='sqrt',
                           min_samples_leaf=0.1,
                           min_samples_split=0.28181818181818186,
                           n_estimators=10, subsample=0.95)
0.7949721953578337


In [20]:
#  Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
classifier =GradientBoostingClassifier(learning_rate=0.2, max_depth=8, max_features='sqrt',
                           min_samples_leaf=0.1,
                           min_samples_split=0.28181818181818186,
                           n_estimators=10, subsample=0.95)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4895 279 1234 635
pod:  0.33975387907972177
pof:  0.053923463471202165
AUC:  0.6429152078042598
accuracy:  0.7851767712622462


In [22]:
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    n_cols = x_train_chi.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        n_cols = x_train_chi.shape[1]
        nn = Sequential()
        nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        nn.add(Dense(250, activation='relu'))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return nn
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

Best: 0.793269 using {'batch_size': 10, 'epochs': 100}
0.735189 (0.047563) with: {'batch_size': 10, 'epochs': 10}
0.792986 (0.002098) with: {'batch_size': 10, 'epochs': 50}
0.793269 (0.003560) with: {'batch_size': 10, 'epochs': 100}
0.758908 (0.019927) with: {'batch_size': 20, 'epochs': 10}
0.776512 (0.019441) with: {'batch_size': 20, 'epochs': 50}
0.778215 (0.022630) with: {'batch_size': 20, 'epochs': 100}
0.735907 (0.005486) with: {'batch_size': 40, 'epochs': 10}
0.772685 (0.018790) with: {'batch_size': 40, 'epochs': 50}
0.762598 (0.029318) with: {'batch_size': 40, 'epochs': 100}
0.742583 (0.006696) with: {'batch_size': 60, 'epochs': 10}
0.676002 (0.103338) with: {'batch_size': 60, 'epochs': 50}
0.662370 (0.123155) with: {'batch_size': 60, 'epochs': 100}
0.599139 (0.187533) with: {'batch_size': 80, 'epochs': 10}
0.686468 (0.143346) with: {'batch_size': 80, 'epochs': 50}
0.671456 (0.137360) with: {'batch_size': 80, 'epochs': 100}
0.736901 (0.004456) with: {'batch_size': 100, 'epochs':

In [23]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

import time
start = time.time()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2,  epochs=100, batch_size= 20, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/100
254/254 [==============================] - 5s 17ms/step - loss: 11.0715 - accuracy: 0.5869 - val_loss: 0.9696 - val_accuracy: 0.2950
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.0970 - accuracy: 0.6205 - val_loss: 0.5980 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 2ms/step - loss: 0.6383 - accuracy: 0.7168 - val_loss: 0.5451 - val_accuracy: 0.7579
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5671 - accuracy: 0.7258 - val_loss: 0.5168 - val_accuracy: 0.7681
Epoch 5/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5505 - accuracy: 0.7324 - val_loss: 0.5020 - val_accuracy: 0.7390
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5448 - accuracy: 0.7319 - val_loss: 0.6756 - val_accuracy: 0.5836
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5576 - accuracy: 0.7281 - val_loss: 0.5445 - val_accuracy: 0.73

Epoch 1/100
254/254 [==============================] - 2s 4ms/step - loss: 11.8292 - accuracy: 0.6062 - val_loss: 0.6968 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.0879 - accuracy: 0.6281 - val_loss: 0.7854 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6655 - accuracy: 0.7089 - val_loss: 0.6001 - val_accuracy: 0.7287
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6567 - accuracy: 0.6990 - val_loss: 0.8303 - val_accuracy: 0.7287
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6110 - accuracy: 0.7180 - val_loss: 0.6120 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5945 - accuracy: 0.7323 - val_loss: 0.5861 - val_accuracy: 0.7279
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6006 - accuracy: 0.7153 - val_loss: 0.6349 - val_accuracy: 0.727

254/254 [==============================] - 3s 6ms/step - loss: 10.0647 - accuracy: 0.6181 - val_loss: 1.6353 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 4ms/step - loss: 0.9186 - accuracy: 0.6586 - val_loss: 0.7248 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.7477 - accuracy: 0.6848 - val_loss: 0.5601 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5729 - accuracy: 0.7270 - val_loss: 0.6466 - val_accuracy: 0.7326
Epoch 5/100
254/254 [==============================] - 1s 5ms/step - loss: 0.5593 - accuracy: 0.7245 - val_loss: 0.4755 - val_accuracy: 0.7768
Epoch 6/100
254/254 [==============================] - 1s 6ms/step - loss: 0.5225 - accuracy: 0.7431 - val_loss: 0.4771 - val_accuracy: 0.7382
Epoch 7/100
254/254 [==============================] - 2s 7ms/step - loss: 0.5071 - accuracy: 0.7475 - val_loss: 0.4578 - val_accuracy: 0.7926
Epoch 8/10

Epoch 1/100
254/254 [==============================] - 16s 27ms/step - loss: 10.2016 - accuracy: 0.6248 - val_loss: 1.5936 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 3s 10ms/step - loss: 0.8730 - accuracy: 0.6614 - val_loss: 0.5332 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 2s 9ms/step - loss: 0.7569 - accuracy: 0.7064 - val_loss: 0.5056 - val_accuracy: 0.7445
Epoch 4/100
254/254 [==============================] - 3s 11ms/step - loss: 0.5773 - accuracy: 0.7264 - val_loss: 0.5377 - val_accuracy: 0.7271
Epoch 5/100
254/254 [==============================] - 3s 12ms/step - loss: 0.5592 - accuracy: 0.7353 - val_loss: 0.4922 - val_accuracy: 0.7271
Epoch 6/100
254/254 [==============================] - 3s 10ms/step - loss: 0.5419 - accuracy: 0.7327 - val_loss: 0.5426 - val_accuracy: 0.7271
Epoch 7/100
254/254 [==============================] - 3s 10ms/step - loss: 0.5474 - accuracy: 0.7223 - val_loss: 0.4693 - val_accuracy

Epoch 1/100
254/254 [==============================] - 9s 17ms/step - loss: 11.8718 - accuracy: 0.6079 - val_loss: 1.2941 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 2s 8ms/step - loss: 0.9589 - accuracy: 0.6478 - val_loss: 0.6792 - val_accuracy: 0.5363
Epoch 3/100
254/254 [==============================] - 2s 8ms/step - loss: 0.6975 - accuracy: 0.6935 - val_loss: 0.6786 - val_accuracy: 0.5363
Epoch 4/100
254/254 [==============================] - 2s 8ms/step - loss: 0.6286 - accuracy: 0.7013 - val_loss: 0.5874 - val_accuracy: 0.7271
Epoch 5/100
254/254 [==============================] - 2s 9ms/step - loss: 0.5992 - accuracy: 0.7261 - val_loss: 0.6187 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 2s 9ms/step - loss: 0.6053 - accuracy: 0.7269 - val_loss: 0.6567 - val_accuracy: 0.5852
Epoch 7/100
254/254 [==============================] - 2s 7ms/step - loss: 0.6107 - accuracy: 0.7233 - val_loss: 0.5448 - val_accuracy: 0.72

Epoch 1/100
254/254 [==============================] - 21s 35ms/step - loss: 13.4586 - accuracy: 0.6069 - val_loss: 1.2658 - val_accuracy: 0.3115
Epoch 2/100
254/254 [==============================] - 3s 10ms/step - loss: 0.9046 - accuracy: 0.6652 - val_loss: 0.5884 - val_accuracy: 0.7287
Epoch 3/100
254/254 [==============================] - 2s 9ms/step - loss: 0.6656 - accuracy: 0.7118 - val_loss: 0.6346 - val_accuracy: 0.7311
Epoch 4/100
254/254 [==============================] - 2s 10ms/step - loss: 0.6037 - accuracy: 0.7380 - val_loss: 0.5784 - val_accuracy: 0.7287
Epoch 5/100
254/254 [==============================] - 2s 9ms/step - loss: 0.5959 - accuracy: 0.7231 - val_loss: 0.5782 - val_accuracy: 0.7295
Epoch 6/100
254/254 [==============================] - 3s 10ms/step - loss: 0.5785 - accuracy: 0.7369 - val_loss: 0.5830 - val_accuracy: 0.7295
Epoch 7/100
254/254 [==============================] - 2s 9ms/step - loss: 0.5754 - accuracy: 0.7439 - val_loss: 0.5819 - val_accuracy: 

Epoch 1/100
254/254 [==============================] - 26s 26ms/step - loss: 14.1722 - accuracy: 0.5987 - val_loss: 1.1208 - val_accuracy: 0.3068
Epoch 2/100
254/254 [==============================] - 2s 9ms/step - loss: 0.9472 - accuracy: 0.6384 - val_loss: 0.5936 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 2s 9ms/step - loss: 0.6109 - accuracy: 0.7205 - val_loss: 1.1052 - val_accuracy: 0.3502
Epoch 4/100
254/254 [==============================] - 2s 10ms/step - loss: 0.6663 - accuracy: 0.6892 - val_loss: 0.5275 - val_accuracy: 0.7287
Epoch 5/100
254/254 [==============================] - 2s 9ms/step - loss: 0.5949 - accuracy: 0.7170 - val_loss: 0.6573 - val_accuracy: 0.7295
Epoch 6/100
254/254 [==============================] - 3s 10ms/step - loss: 0.5753 - accuracy: 0.7329 - val_loss: 0.5466 - val_accuracy: 0.7326
Epoch 7/100
254/254 [==============================] - 2s 9ms/step - loss: 0.5255 - accuracy: 0.7426 - val_loss: 0.4776 - val_accuracy: 0

Epoch 1/100
254/254 [==============================] - 16s 23ms/step - loss: 12.8117 - accuracy: 0.5966 - val_loss: 1.4467 - val_accuracy: 0.3036
Epoch 2/100
254/254 [==============================] - 2s 10ms/step - loss: 0.8328 - accuracy: 0.6457 - val_loss: 0.5821 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 2s 9ms/step - loss: 0.6860 - accuracy: 0.7101 - val_loss: 0.7702 - val_accuracy: 0.4345
Epoch 4/100
254/254 [==============================] - 2s 10ms/step - loss: 0.6897 - accuracy: 0.6902 - val_loss: 0.5834 - val_accuracy: 0.7571
Epoch 5/100
254/254 [==============================] - 2s 10ms/step - loss: 0.6043 - accuracy: 0.7182 - val_loss: 0.6469 - val_accuracy: 0.7279
y2_pred:  [[1.61039531e-01]
 [1.04326695e-01]
 [6.65995181e-02]
 [1.98733389e-01]
 [7.62856901e-02]
 [5.71682155e-02]
 [2.93844253e-01]
 [1.68147057e-01]
 [8.16888809e-02]
 [1.43849075e-01]
 [1.64218485e-01]
 [1.37229979e-01]
 [1.25385463e-01]
 [1.77519083e-01]
 [6.18301034e-02]

Epoch 1/100
254/254 [==============================] - 14s 17ms/step - loss: 7.6026 - accuracy: 0.6075 - val_loss: 0.6067 - val_accuracy: 0.7248
Epoch 2/100
254/254 [==============================] - 2s 9ms/step - loss: 0.8367 - accuracy: 0.6535 - val_loss: 0.6622 - val_accuracy: 0.7248
Epoch 3/100
254/254 [==============================] - 3s 10ms/step - loss: 0.6653 - accuracy: 0.7084 - val_loss: 0.8936 - val_accuracy: 0.4109
Epoch 4/100
254/254 [==============================] - 3s 11ms/step - loss: 0.6361 - accuracy: 0.6990 - val_loss: 0.7069 - val_accuracy: 0.7256
y2_pred:  [[7.76504576e-02]
 [3.88439000e-02]
 [8.47385228e-02]
 [3.83862257e-02]
 [6.00471795e-02]
 [2.38658458e-01]
 [3.55276287e-01]
 [5.10959923e-02]
 [3.62067580e-01]
 [2.28948891e-02]
 [9.79575813e-02]
 [7.75846839e-02]
 [1.28045917e-01]
 [2.20479548e-01]
 [4.16663289e-02]
 [1.67760551e-01]
 [1.09196603e-01]
 [9.08247828e-02]
 [7.49943554e-02]
 [2.91980505e-02]
 [2.02941477e-01]
 [8.59133899e-02]
 [2.57515907e-01]


Epoch 1/100
254/254 [==============================] - 7s 14ms/step - loss: 16.7268 - accuracy: 0.6197 - val_loss: 1.0083 - val_accuracy: 0.2973
Epoch 2/100
254/254 [==============================] - 2s 10ms/step - loss: 1.0350 - accuracy: 0.6389 - val_loss: 1.0164 - val_accuracy: 0.7248
Epoch 3/100
254/254 [==============================] - 3s 10ms/step - loss: 0.8060 - accuracy: 0.6741 - val_loss: 0.9622 - val_accuracy: 0.3730
Epoch 4/100
254/254 [==============================] - 3s 10ms/step - loss: 0.7052 - accuracy: 0.6815 - val_loss: 0.5950 - val_accuracy: 0.7248
Epoch 5/100
254/254 [==============================] - 2s 10ms/step - loss: 0.7290 - accuracy: 0.6923 - val_loss: 0.7471 - val_accuracy: 0.7256
Epoch 6/100
254/254 [==============================] - 3s 10ms/step - loss: 0.6188 - accuracy: 0.7235 - val_loss: 0.5853 - val_accuracy: 0.7256
Epoch 7/100
254/254 [==============================] - 3s 10ms/step - loss: 0.5740 - accuracy: 0.7395 - val_loss: 0.5864 - val_accuracy

In [ ]:
##RNN Hyperparemeter tuning
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    #n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        #n_cols = X2_train.shape[1]
        #nn = Sequential()
        #nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        #nn.add(Dense(250, activation='relu'))
        #nn.add(Dense(1, activation='sigmoid'))
        #nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model = Sequential()
        model.add(Embedding(10000, 32))
        model.add(LSTM(32))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
        return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

In [24]:
#RNN
import time
start = time.time()

def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=10, batch_size=100, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/10
51/51 [==============================] - 7s 28ms/step - loss: 0.6135 - acc: 0.6969 - val_loss: 0.5156 - val_acc: 0.7287
Epoch 2/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4990 - acc: 0.7350 - val_loss: 0.4909 - val_acc: 0.7453
Epoch 3/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4534 - acc: 0.7701 - val_loss: 0.4785 - val_acc: 0.7737
Epoch 4/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4557 - acc: 0.7846 - val_loss: 0.4899 - val_acc: 0.7626
Epoch 5/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4451 - acc: 0.7855 - val_loss: 0.4639 - val_acc: 0.7776
Epoch 6/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4390 - acc: 0.7912 - val_loss: 0.4586 - val_acc: 0.7744
Epoch 7/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4165 - acc: 0.7996 - val_loss: 0.4599 - val_acc: 0.7815
Epoch 8/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4051 - a

Epoch 1/10
51/51 [==============================] - 3s 28ms/step - loss: 0.6058 - acc: 0.7334 - val_loss: 0.5194 - val_acc: 0.7287
Epoch 2/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4936 - acc: 0.7407 - val_loss: 0.4831 - val_acc: 0.7476
Epoch 3/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4667 - acc: 0.7656 - val_loss: 0.4766 - val_acc: 0.7721
Epoch 4/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4584 - acc: 0.7851 - val_loss: 0.4816 - val_acc: 0.7658
Epoch 5/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4464 - acc: 0.7858 - val_loss: 0.4624 - val_acc: 0.7729
Epoch 6/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4291 - acc: 0.7970 - val_loss: 0.4618 - val_acc: 0.7744
Epoch 7/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4171 - acc: 0.8085 - val_loss: 0.4592 - val_acc: 0.7752
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.3987 - a

Epoch 1/10
51/51 [==============================] - 3s 29ms/step - loss: 0.6006 - acc: 0.7293 - val_loss: 0.5104 - val_acc: 0.7326
Epoch 2/10
51/51 [==============================] - 1s 18ms/step - loss: 0.5010 - acc: 0.7401 - val_loss: 0.4969 - val_acc: 0.7445
Epoch 3/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4542 - acc: 0.7747 - val_loss: 0.4780 - val_acc: 0.7618
Epoch 4/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4526 - acc: 0.7858 - val_loss: 0.4695 - val_acc: 0.7760
Epoch 5/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4415 - acc: 0.7896 - val_loss: 0.4623 - val_acc: 0.7776
Epoch 6/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4281 - acc: 0.8028 - val_loss: 0.4691 - val_acc: 0.7776
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4070 - acc: 0.8088 - val_loss: 0.4810 - val_acc: 0.7689
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.3989 - a

Epoch 1/10
51/51 [==============================] - 3s 28ms/step - loss: 0.5973 - acc: 0.6998 - val_loss: 0.5066 - val_acc: 0.7382
Epoch 2/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4926 - acc: 0.7481 - val_loss: 0.4916 - val_acc: 0.7587
Epoch 3/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4455 - acc: 0.7814 - val_loss: 0.4740 - val_acc: 0.7744
Epoch 4/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4444 - acc: 0.7908 - val_loss: 0.4704 - val_acc: 0.7792
Epoch 5/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4404 - acc: 0.7918 - val_loss: 0.4634 - val_acc: 0.7784
Epoch 6/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4278 - acc: 0.7936 - val_loss: 0.4589 - val_acc: 0.7808
Epoch 7/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4103 - acc: 0.8123 - val_loss: 0.4691 - val_acc: 0.7689
Epoch 8/10
51/51 [==============================] - 1s 17ms/step - loss: 0.3954 - a

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.5976 - acc: 0.7279 - val_loss: 0.5086 - val_acc: 0.7232
Epoch 2/10
51/51 [==============================] - 1s 15ms/step - loss: 0.4855 - acc: 0.7449 - val_loss: 0.4920 - val_acc: 0.7618
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4422 - acc: 0.7806 - val_loss: 0.4767 - val_acc: 0.7831
Epoch 4/10
51/51 [==============================] - 1s 13ms/step - loss: 0.4426 - acc: 0.7947 - val_loss: 0.4731 - val_acc: 0.7855
Epoch 5/10
51/51 [==============================] - 1s 14ms/step - loss: 0.4358 - acc: 0.7930 - val_loss: 0.4869 - val_acc: 0.7689
Epoch 6/10
51/51 [==============================] - 1s 14ms/step - loss: 0.4118 - acc: 0.8090 - val_loss: 0.4699 - val_acc: 0.7815
Epoch 7/10
51/51 [==============================] - 1s 13ms/step - loss: 0.3994 - acc: 0.8195 - val_loss: 0.4616 - val_acc: 0.7855
Epoch 8/10
51/51 [==============================] - 1s 15ms/step - loss: 0.3767 - a